In [1]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 67.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 236.8/236.8 kB 32.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 57.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 71.1 MB/s eta 0:00:00


In [2]:
import pandas as pd
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split
from transformers import TFAutoModel, AutoTokenizer

In [4]:
df = pd.read_csv('/content/olid-training-v1.0.tsv', sep='\t')

# Remove unnecessary columns
df = df.drop(['id', 'subtask_b', 'subtask_c'], axis=1)

# Encode the labels
labels = {'NOT': 0, 'OFF': 1}
df['subtask_a'] = df['subtask_a'].apply(lambda x: labels[x])

In [5]:
# Split the data into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(df['tweet'], df['subtask_a'], test_size=0.2, random_state=42)

In [6]:
# Load the BERT tokenizer
tokenizer = AutoTokenizer.from_pretrained('bert-base-uncased')

In [7]:
# Tokenize the training and validation sets
train_encodings = tokenizer(list(X_train), truncation=True, padding=True)
val_encodings = tokenizer(list(X_val), truncation=True, padding=True)

In [8]:
# Create TensorFlow datasets
train_dataset = tf.data.Dataset.from_tensor_slices((
    dict(train_encodings),
    y_train.values
))

val_dataset = tf.data.Dataset.from_tensor_slices((
    dict(val_encodings),
    y_val.values
))

In [9]:
# Batch the datasets
train_dataset = train_dataset.batch(16)
val_dataset = val_dataset.batch(16)

In [10]:
# Define the base model
base_model = TFAutoModel.from_pretrained('bert-base-uncased')

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFBertModel: ['cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing TFBertModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFBertModel were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions w

In [11]:
# Define the meta-learner model
meta_model = tf.keras.Sequential([
    tf.keras.layers.Input(shape=(768,)),
    tf.keras.layers.Dense(512, activation='relu'),
    tf.keras.layers.Dense(256, activation='relu'),
    tf.keras.layers.Dense(2, activation='softmax')
])

In [12]:
# Model summary
meta_model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 512)               393728    
                                                                 
 dense_1 (Dense)             (None, 256)               131328    
                                                                 
 dense_2 (Dense)             (None, 2)                 514       
                                                                 
Total params: 525,570
Trainable params: 525,570
Non-trainable params: 0
_________________________________________________________________


In [13]:
# Define the loss function and optimizer
loss_fn = tf.keras.losses.SparseCategoricalCrossentropy()
optimizer = tf.keras.optimizers.Adam(learning_rate=1e-5)

In [14]:
# Define the training loop
@tf.function
def train_step(inputs, labels):
    with tf.GradientTape() as tape:
        base_output = base_model(inputs)['last_hidden_state'][:, 0, :]
        meta_output = meta_model(base_output)
        loss = loss_fn(labels, meta_output)
    gradients = tape.gradient(loss, meta_model.trainable_weights)
    optimizer.apply_gradients(zip(gradients, meta_model.trainable_weights))
    return loss

In [15]:
# Train the model for more epochs
for epoch in range(20):
    for inputs, labels in train_dataset:
        loss = train_step(inputs, labels)
    print('Epoch', epoch+1, '- Loss:', loss.numpy())

Epoch 1 - Loss: 0.4840094
Epoch 2 - Loss: 0.42743176
Epoch 3 - Loss: 0.39241284
Epoch 4 - Loss: 0.36905897
Epoch 5 - Loss: 0.35461152
Epoch 6 - Loss: 0.3446444
Epoch 7 - Loss: 0.33797327
Epoch 8 - Loss: 0.33225483
Epoch 9 - Loss: 0.3279613
Epoch 10 - Loss: 0.3240325
Epoch 11 - Loss: 0.32091752
Epoch 12 - Loss: 0.31784087
Epoch 13 - Loss: 0.31584692
Epoch 14 - Loss: 0.31311902
Epoch 15 - Loss: 0.31120914
Epoch 16 - Loss: 0.30940607
Epoch 17 - Loss: 0.30823597
Epoch 18 - Loss: 0.30631262
Epoch 19 - Loss: 0.30444115
Epoch 20 - Loss: 0.3022979


In [16]:
# Evaluate the model on the validation set
true_labels = []
predictions = []
for inputs, labels in val_dataset:
    base_output = base_model(inputs)['last_hidden_state'][:, 0, :]
    meta_output = meta_model(base_output).numpy()
    predicted_labels = np.argmax(meta_output, axis=1)
    true_labels.extend(labels.numpy())
    predictions.extend(predicted_labels)

from sklearn.metrics import classification_report

# Print the classification report
print(classification_report(true_labels, predictions))

              precision    recall  f1-score   support

           0       0.77      0.90      0.83      1733
           1       0.71      0.48      0.58       915

    accuracy                           0.75      2648
   macro avg       0.74      0.69      0.70      2648
weighted avg       0.75      0.75      0.74      2648

